# Bermudan Swaptions Valuation
Valuation of semi-annual fixed & floating leg bermudan swaptions under the LSM framework using Neural Net and XGBoost instead of least-squares to compute the conditional expected continuation value at each time step. Simulated rates used for valuing the bermudan swaptions are obtained from "main_LMM_SABR" and the results from that is stored under the "simulation_results".

The example in the main_LMM_SABR notebook only generates 1k simulated paths. \
10k simulated paths are given in the folder as an example by running the main_LMM_SABR and changing the number of simulations parameter.

## Need:
- simulated zeros paths (from main_LMM_SABR) \
generate the amount of paths you want and load the paths here
- berm_swaptions class (from berm_swaptions.py)

In [12]:
#load required libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso
from xgboost import XGBRegressor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR, ReduceLROnPlateau
from torch.utils.data import random_split, DataLoader
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import scipy
from sklearn import model_selection, metrics
import os
from functools import partial
from datetime import date
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import pickle
import time

In [15]:
from berm_swaption_class import * #import berm_swaptions & berm_swaptions_power

In [43]:
'''
load rates sample dataset
'''
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as handle:
        return pickle.load(handle)

path_rates = os.path.join(os.getcwd(), "simulation_results")
path_valuation_results = os.path.join(os.getcwd(), "valuation_results")
path_tune_chk = os.path.join(os.getcwd(), "tune_checkpoint") #main path for tune checkpoint and trial results

In [ ]:
sim_rates_1k = load_object(path_rates+"\\simulated_zeros_1k.pkl")
sim_rates_10k = load_object(path_rates+"\\simulated_zeros_10k.pkl")

In [30]:
#check sample rate paths
print(sim_rates_1k.shape)
print(sim_rates_1k[:5,:5,0])

print(sim_rates_10k.shape)
print(sim_rates_10k[:5,:5,0])

(20, 20, 1000)
[[0.99884244 0.         0.         0.         0.        ]
 [0.9976228  0.99854474 0.         0.         0.        ]
 [0.99739711 0.99706474 0.99810346 0.         0.        ]
 [0.99730355 0.99641432 0.99606452 0.99800189 0.        ]
 [0.99679304 0.99587095 0.99580402 0.99592028 0.99851665]]
(20, 20, 10000)
[[0.99884244 0.         0.         0.         0.        ]
 [0.9976228  0.99877373 0.         0.         0.        ]
 [0.99739711 0.99751846 0.99862225 0.         0.        ]
 [0.99730355 0.99725279 0.99712725 0.99849212 0.        ]
 [0.99679304 0.99639462 0.99583648 0.99571872 0.99731072]]


In [31]:
#create list containing all the rates
rate_list = [sim_rates_1k, sim_rates_10k]

### Create NN Model
Changeable number of layers/neurons for Bayesian Opt

In [18]:
# =============================================================================
# define custom neural net class for bayes opt tuning
# =============================================================================
class net(nn.Module):
    def __init__(self, input_dim, neurons, hidden_layers, activation_fn):
        super(net, self).__init__()
        self.activation_fn = activation_fn
        self.input_dim = input_dim
        
        self.layers= nn.ModuleList()
        #add first layer
        self.layers.append(nn.Linear(input_dim, neurons))
        
        for k in range(1, hidden_layers+1):
            self.layers.append(nn.Linear(neurons, neurons))
            
        #output layer
        self.layers.append(nn.Linear(neurons, 1))
        

    def forward(self, x):
        for i in range(len(self.layers)-1): #except last layer
            x = self.layers[i](x)
            x = self.activation_fn(x)

        #output layer (no activation)
        x = self.layers[-1](x)
        return x

In [4]:
#laguerre polynomials for the basis functions at each step

def laguerre_scipy(x, k):
    #x: paths x nvars
    #return paths x (nvars * polynomials)
    
    dim = x.shape
    L = np.zeros((dim[0], 1)) #paths x 1 #placeholder
    
    for i in range(dim[1]):
        weight = np.exp(-x[:, i]/2)
        for j in range(k):
            L = np.concatenate([L,
                            (weight.reshape(-1,1) * 
                            scipy.special.eval_genlaguerre(j, 0, x[:,i]).reshape(-1,1))], axis = 1)
    
    L = L[:,1:]
    return(L)


Define training NN function using tune to report validation losses for early stopping & bayes-optimization

In [66]:
def trainNN(config, x, y, input_size, max_num_epochs, use_tune = True,
            checkpoint_dir = None):
    '''
    config = dict of number of neurons, num hidden, activation function used, learning rate, batch size
    '''
    
    model = net(input_size, config["num_neurons"], config["num_hidden"], config["activation_fn"])
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    
    criterion = torch.nn.functional.mse_loss
    optimizer=optim.Adam(model.parameters(),lr= config["lr"])
    
    lr_scheduler = ReduceLROnPlateau(optimizer, mode = 'min',
                                     patience = 10, threshold_mode = 'rel', threshold = 1e-4)
    
    
    x = x.reshape(-1, input_size)
    y = y.reshape(-1,1)
    
    #split
    x_train,x_test,y_train,y_test = model_selection.train_test_split(x,
                                                                    y, test_size=0.1) 
    
    x_train = torch.tensor(x_train,dtype=torch.float)
    y_train = torch.tensor(y_train,dtype=torch.float)
    x_test = torch.tensor(x_test,dtype=torch.float)
    y_test = torch.tensor(y_test,dtype=torch.float)
    
    
    train_dataset = torch.utils.data.TensorDataset(x_train.view(-1, input_size), y_train.view(-1,1))
    
    train_loader = torch.utils.data.DataLoader(dataset = train_dataset, 
                                               batch_size= 32,  #
                                               shuffle=True)

    test_dataset = torch.utils.data.TensorDataset(x_test.view(-1, input_size), y_test.view(-1,1))
    
    test_loader = torch.utils.data.DataLoader(dataset = test_dataset, 
                                               batch_size= 32,  #
                                               shuffle=True)


    #track losses
    train_losses, test_losses = [], []
    test_r2 = []
    avg_train_losses, avg_test_losses  = [], []
    avg_test_r2  = []
    
    #set initial losses as inf
    avg_train_losses.append(float('inf'))
    avg_test_losses.append(float('inf'))
    
    def accuracy_r2(y_true, y_pred):
        #for tensors
        y_true = y_true.detach().numpy().reshape(-1,1)
        y_pred = y_pred.detach().numpy().reshape(-1,1)
        return metrics.r2_score(y_true, y_pred)
    
    for epoch in range(max_num_epochs):
        # =======================
        # train        
        # =======================
        model.train() #training mode    
        for i, (features, price) in enumerate(train_loader):
            optimizer.zero_grad()
            features, price= features.to(device), price.to(device)
            
            #forward
            model_output = model(features)
            
            #backprop
            loss = criterion(model_output,price)
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
        
            
        # ======================
        # eval
        # ======================
        model.eval() # prep model for evaluation
        for i, (features, price) in enumerate(test_loader):
            with torch.no_grad():
                features = features.to(device)
                price = price.to(device)
                # forward pass
                model_output = model(features)
                
                # calculate the loss
                loss = criterion(model_output, price)
                # record validation loss
                test_losses.append(loss.item())
        

        
        train_loss, test_loss = np.mean(train_losses), np.mean(test_losses)
        

        #whole validation set
        test_r2 = accuracy_r2(test_dataset.tensors[1], model(test_dataset.tensors[0]))
    
        avg_train_losses.append(train_loss)
        avg_test_losses.append(test_loss)
        avg_test_r2.append(test_r2)
        
        #lr scheduler min mse
        lr_scheduler.step(test_loss)
        
        
        if use_tune:
            print("Saving checkpoint")
            with tune.checkpoint_dir(epoch) as checkpoint_dir:
                  path = os.path.join(checkpoint_dir, "checkpoint")
                  torch.save(model.state_dict(), path) # optimizer.state_dict()
                  
            tune.report(loss= test_loss, accuracy= test_r2) #report back to tune
            
            
            
        #clear list
        train_losses = []
        test_losses = []
        

    print("Finished Training")    

#### Define main NN function that uses tune with ASHAscheduler and BayesianOpt from skopt to try different nn configs

In [67]:
# =============================================================================
# setup environament for ray tune
# =============================================================================
'''
config: dictionary of search space
num_neurons = #of neurons for each hidden layer
num_hidden = number of hidden layers (not incl. output layer)
activation_fn = activation function for each hidden layer
batch size = batch size for mini batch GD
lr = learning rate for adam
'''
def main_NN(data, num_samples=15, max_num_epochs=30,
            metric = 'loss', mode = 'min',
            checkpoint_dir = None,
            trial_path = None,
            experiment_name = "experiment"+str(date.today()),
            trial_name = None):
    '''
    data = tuple of (X, y) dataset used for training and validation
    num_samples = samples to search from search space
    max_num_epochs = max number of epochs to train the NN
    
    search over NN hyperspace defined by config
    max_num_epochs = max epochs for ASHAScheduler to terminate training
    num_samples = num trials
    
    trial_name= current trial name
    trial_dir = same as trial name
    
    '''
    #if not using bayes-opt
    
    # config = {
    #     "num_neurons": tune.choice([16, 32, 64, 128]),
    #     "num_hidden": tune.choice([2,3,4]),
    #     "activation_fn" : tune.choice([F.relu, F.leaky_relu]),
    #     "batch_size": tune.choice([32]),
    #     "lr": tune.loguniform(1e-4, 1e-1)
    # }
    
    bayes_searchspace = {
        "num_neurons": Categorical([int(x) for x in  2**np.arange(4,8)]),
        "num_hidden": Integer(2, 4, 'uniform'),
        "activation_fn" : Categorical([F.relu]),
        # "batch_size": Integer(16, 32),#Categorical([int(16), int(32)]),
        "lr": Real(1e-4, 1e-2, 'log-uniform')
        }
    
    
    skopt_search = SkOptSearch(space = bayes_searchspace, metric="accuracy", mode="max")
    
    
    '''
    can set metric/mode in scheduler or tune.run
    ASHA scheduler can set max_num_epochs
    grace_period = min number of iterations before stopping
    '''
    scheduler = ASHAScheduler(
                    metric= metric, #loss
                    mode= mode, #min
                    time_attr='training_iteration',
                    max_t=max_num_epochs,
                    grace_period=10, #set grace period before stopping here
                    reduction_factor=2)
    
    '''
    CLIReporter for python console
    what to print to console
    '''
    reporter = CLIReporter( 
        parameter_columns=["lr", "num_neurons", "num_hidden"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    
    #get dataset
    x, y = data
    input_size = x.shape[1]
    
    
    def trial_name_string(trial):
        return trial_name+str(trial.trial_id)
    
    result = tune.run(
            partial(trainNN, x = x, y = y, input_size = input_size, max_num_epochs = max_num_epochs,
                    checkpoint_dir = checkpoint_dir),
            resources_per_trial={"cpu": 3, "gpu": 0},
            # config=config,
            search_alg = skopt_search,
            num_samples=num_samples,
            scheduler=scheduler,
            reuse_actors = True,
            progress_reporter=reporter,
            name= experiment_name,
            local_dir = trial_path,
            trial_name_creator = trial_name_string,
            trial_dirname_creator = trial_name_string)
    
    #get best trial
    best_trial = result.get_best_trial(metric, mode, "last") #"accuracy" , max
    
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))
    
    best_trained_model = net(input_size, best_trial.config["num_neurons"],
                             best_trial.config["num_hidden"], best_trial.config["activation_fn"])
    
    
    #load best model state dict and optimizer from best checkpoint path
    best_checkpoint_dir = best_trial.checkpoint.value
    
    model_state  = torch.load(os.path.join( #optimizer_state
         checkpoint_dir, "checkpoint"))
    
    
    best_trained_model.load_state_dict(model_state)

    
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        # if gpus_per_trial > 1:
        #     best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)
    
    
    return best_trained_model, result

## Bermudan Swaption Valuations with Neural Network
Set up environment and train hyperparameters for nn using config
Use simulated interest rate model
loop through exercise steps to calculate continuation value

In [68]:
'''
wrapper function for neural network framework to calculate out-of-sample results
'''
def oos_nn(berm_swaptions, model, step, laguerre = False):
    '''
    berm_swaptions = OOS bermudan swaptions class
    model = neural net model
    '''
    itm = np.where(berm_swaptions.intrinsic[:, step] > 0)[0]
    
    X = berm_swaptions.X(step)[0]
    initial_dim = X.shape
    
    X = X[itm, :]
    
    if laguerre:
        X = laguerre_scipy(X, 3)
    
    y = berm_swaptions.value[:, step+1]
    y = berm_swaptions.params["rate_matrix"][step, step, itm] * y[itm] #discount to now
    
    pred = model(torch.tensor(X, dtype = torch.float)).detach().numpy().reshape(-1,1)
    
    pred_all = np.zeros((initial_dim[0], 1))
    pred_all[itm, :] = pred
    
    #update
    berm_swaptions.update(step, pred_all.ravel())
    
    #oos r2
    r2_score = metrics.r2_score(pred.reshape(-1,1), y.reshape(-1,1))
    return(r2_score)

In [69]:
'''
main valuation function for neural network
'''
def berm_nn(berm_swaptions, sim_rates, strike, lockout, tenor, opt_type = "rec", max_num_epochs = 30,
                num_samples = 10, metric = 'loss', mode = 'min',
                test_size = 0.1, seed = 123, trial_path = None, checkpoint_dir = None):
    '''
    berm_swaptions: class
    max_num_epochs = maximum number of epochs before stopping
    num_samples = number of samples taken for bayesopt
    opt_type = "rec"/"pay" for receiver or payer
    sim_rates : simulated zeros matrix n x n x paths
    strike: strike for the swaption
    lockout: in years
    tenor: in years
    test_size: float to split the dataset to training-testing split
    seed: seed for splitting
    '''
    total_dim = sim_rates.shape
    total_paths = total_dim[2] #no paths
    
    testing_paths = int(test_size * total_paths)
    training_paths = total_paths - testing_paths
    
    np.random.seed(123)
    testing_idx= np.sort(np.random.choice(np.arange(total_paths), testing_paths ,replace = False))
    training_idx = np.setdiff1d(np.arange(total_paths), testing_idx)
    
    sim = berm_swaptions(sim_rates[:, :, training_idx],
          strike = strike, tenor = tenor, lockout = lockout, opt_type = opt_type)
    
    #out of sample
    sim_oos = berm_swaptions(sim_rates[:, :, testing_idx],
                             strike = strike, tenor = tenor, lockout = lockout, opt_type = opt_type)
    
    steps = sim.exercisable_steps
    steps = np.flip(steps)[1:]
      
    tune_result_list = []
    model_list = []
    
    best_results_r2 = {"Step":[], "Accuracy":[]}
    oos_r2 = {"Step":[], "Accuracy":[]}
    
    sampling_increase = 0#np.cumsum(steps%3)
    total_time = 0
    ct = 0
    for i in steps:
        '''
        #at each time step t_{M-1} compute V_{t_{M-1}} = max(h_{tm-1}, Q_{tm-1})
        #where Q is the expected continuation value
        #V will be used as an output for training the NN
        
        #first calculate Q at tm-1
        #which is the discounted value of G at tm
        #G is the approximated function from the neural network    
        '''
        itm = np.where(sim.intrinsic[:, i] > 0)[0]
        
        X = sim.X(i)[0]
        X = X[itm, :]
        
        X = laguerre_scipy(X, 3)
        
        y = sim.value[:, i+1]
        y = sim.params["rate_matrix"][i, i, itm] * y[itm] #discount to now
        
        t0 = time.time()
        #===================================================================================
        # NN model
    
   
        model, tune_result = main_NN((X, y),
                                    num_samples = int(num_samples + 0),
                                    max_num_epochs = max_num_epochs,
                           trial_path = trial_path+"\\trials",
                           checkpoint_dir = checkpoint_dir,
                          experiment_name = "nn_step_"+str(i), trial_name = "nn",
                           metric = metric, mode = mode)
        
        tune_result.get_best_trial('loss', 'min', 'last').config
        
        
        tune_result_list.append(tune_result)
        model_list.append(model)
        
        #valuation
        #===================================================================================
        t1 = time.time()
        total_time += (t1 - t0)
        
        pred = model(torch.tensor(X, dtype =  torch.float)).detach().numpy().reshape(-1,1)
        pred_all = np.zeros((training_paths , 1))
        pred_all[itm, :] = pred
        

        best_results_r2["Step"].append(i)
        best_results_r2["Accuracy"].append(metrics.r2_score(pred.reshape(-1,1), y.reshape(-1,1)))
        
        sim.update(i, pred_all.ravel())
        
        print("Step: " + str(i))
        
        # =============================================================================
        # OOS Test
        # =============================================================================
        oos_r2["Step"].append(i)
        
        
        oos_r2["Accuracy"].append(oos_nn(berm_swaptions = sim_oos,
                                           model = model,
                                           step = i, laguerre = True)) #append r2 and update
        ct += 1
        
    
    results_table = pd.DataFrame({"Step": steps,
                                 "Training R2": best_results_r2["Accuracy"],
                                 "OOS R2": oos_r2["Accuracy"],
                                  "Exc Pr": np.sum(sim.index[:, steps], axis = 0)/training_paths,
                                  "Exc Pr OOS": np.sum(sim_oos.index[:, steps], axis = 0)/testing_paths},
                                 )
    print(results_table)
    
    price = np.sum(sim.cmmf[:, :-1] * np.multiply(sim.index[:,1:], sim.value[:,1:]))/training_paths 
    print(price)
    
    price_oos = np.sum(sim_oos.cmmf[:, :-1] * np.multiply(sim_oos.index[:,1:], sim_oos.value[:,1:]))/testing_paths
    print(price_oos)
    
    return (price, price_oos, results_table, total_time, model_list, tune_result_list)

In [70]:
berm_nn(berm_swaptions, sim_rates_1k, strike = 0.0015, lockout = 1, #1 into 10, 0.0015 as strike
                               tenor = 10, opt_type = "rec",
                               num_samples = 3, max_num_epochs = 25, test_size = 0.1, seed = 123, trial_path = path_tune_chk,
                               checkpoint_dir = path_tune_chk)

2020-12-22 00:25:21,718	WARNING experiment.py:274 -- No name detected on trainable. Using DEFAULT.
2020-12-22 00:25:21,718	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.
2020-12-22 00:25:21,774	WARNING trial_runner.py:580 -- Trial Runner checkpointing failed: can't pickle dict_values objects


== Status ==
Memory usage on this node: 11.8/15.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 20.000: None | Iter 10.000: None
Resources requested: 3/12 CPUs, 0/1 GPUs, 0.0/2.93 GiB heap, 0.0/0.98 GiB objects
Result logdir: C:\Users\nhian\Dropbox\My PC (DESKTOP-L6D69LH)\Documents\GitHub\LMM_SABR\tune_checkpoint\trials\nn_step_18
Number of trials: 1/3 (1 RUNNING)
+--------------+----------+-------+------------+---------------+--------------+
| Trial name   | status   | loc   |         lr |   num_neurons |   num_hidden |
|--------------+----------+-------+------------+---------------+--------------|
| nn3b7fe61c   | RUNNING  |       | 0.00391528 |            64 |            2 |
+--------------+----------+-------+------------+---------------+--------------+


Result for nn3b7fe61c:
(pid=44612) Saving checkpoint
(pid=50104) Saving checkpoint
  accuracy: -32.20822828008994
  date: 2020-12-22_00-25-29
  done: false
  experiment_id: 3962d417ccdc4a58a34c74e0507b7f9e
  experiment_tag:

KeyboardInterrupt: 

(pid=None) F1222 00:29:55.508370 26140 39084 redis_client.cc:74]  Check failed: num_attempts < RayConfig::instance().redis_db_connect_retries() Expected 1 Redis shard addresses, found 2
(pid=None) *** Check failure stack trace: ***
(pid=None)     @   00007FF6DCC8052B  public: void __cdecl google::LogMessage::Flush(void) __ptr64
(pid=None)     @   00007FF6DCC7F382  public: __cdecl google::LogMessage::~LogMessage(void) __ptr64
(pid=None)     @   00007FF6DCC51628  public: virtual __cdecl google::NullStreamFatal::~NullStreamFatal(void) __ptr64
(pid=None)     @   00007FF6DCB8E3C4  public: class google::LogMessageVoidify & __ptr64 __cdecl google::LogMessageVoidify::operator=(class google::LogMessageVoidify const & __ptr64) __ptr64
(pid=None)     @   00007FF6DCB8D507  public: class google::LogMessageVoidify & __ptr64 __cdecl google::LogMessageVoidify::operator=(class google::LogMessageVoidify const & __ptr64) __ptr64
(pid=None)     @   00007FF6DCB8CF54  public: class google::LogMessageVoidify

In [ ]:
'''
value bermudan swaptions using NN at each step from the rate_list
'''
nn_results_1_10 = []
for i in rate_list:
    nn_results_1_10.append(berm_nn(berm_swaptions, i, strike = 0.0015, lockout = 1, #1 into 10, 0.0015 as strike
                               tenor = 10, opt_type = "rec",
                               num_samples = 3, max_num_epochs = 25, test_size = 0.1, seed = 123))

## Bermudan Swaption Valuations with Linear Regression
### For Baseline Results

Wrapper function for calculation OOS values for XGB and LR

In [41]:
def oos_pred(berm_swaptions, model, step, laguerre = False):
    '''
    berm_swaptions = OOS bermudan swaptions class
    model = out model with predict method (X,y)
    '''
    itm = np.where(berm_swaptions.intrinsic[:, step] > 0)[0]
    
    X = berm_swaptions.X(step)[0]
    initial_dim = X.shape
    
    X = X[itm, :]
    
    if laguerre:
        X = laguerre_scipy(X, 3)
    
    y = berm_swaptions.value[:, step+1]
    y = berm_swaptions.params["rate_matrix"][step, step, itm] * y[itm] #discount to now
    
    pred = model.predict(X).reshape(-1,1)
    
    pred_all = np.zeros((initial_dim[0], 1))
    pred_all[itm, :] = pred
    
    #update
    berm_swaptions.update(step, pred_all.ravel())
    
    #oos r2
    r2_score = metrics.r2_score(pred.reshape(-1,1), y.reshape(-1,1))
    return(r2_score)


In [35]:
def berm_lr(berm_swaptions, sim_rates, strike, lockout, tenor, opt_type = "rec",
                test_size = 0.1, seed = 123, verbose = False):
    '''
    berm_swaptions: class
    sim_zeros : simulated zeros matrix n x n x paths
    strike: strike for the swaption
    lockout: in years
    tenor: in years
    test_size: float to split the dataset to training-testing split
    seed: seed for splitting
    '''
    verboseprint = print if verbose else lambda *a, **k: None
    
    total_dim = sim_rates.shape
    total_paths = total_dim[2] #no paths
    
    testing_paths = int(test_size * total_paths)
    training_paths = total_paths - testing_paths
    
    np.random.seed(123)
    testing_idx= np.sort(np.random.choice(np.arange(total_paths), testing_paths ,replace = False))
    training_idx = np.setdiff1d(np.arange(total_paths), testing_idx)
    
    sim = berm_swaptions(sim_rates[:, :, training_idx],
          strike = strike, tenor = tenor, lockout = lockout, opt_type = opt_type)
    
    #out of sample
    sim_oos = berm_swaptions(sim_rates[:, :, testing_idx],
                             strike = strike, tenor = tenor, lockout = lockout, opt_type = opt_type)
    
    steps = sim.exercisable_steps
    steps = np.flip(steps)[1:]
      
    
    best_results_r2 = {"Step":[], "Accuracy":[]}
    oos_r2 = {"Step":[], "Accuracy":[]}
    
    total_time = 0
    for i in steps:
        '''
        #at each time step t_{M-1} compute V_{t_{M-1}} = max(h_{tm-1}, Q_{tm-1})
        #where Q is the expected continuation value
        #V will be used as an output for training the NN
        
        #first calculate Q at tm-1
        #which is the discounted value of G at tm
        #G is the approximated function from the neural network    
        '''
        itm = np.where(sim.intrinsic[:, i] > 0)[0]
        
        X = sim.X(i)[0]
        X = X[itm, :]
        
        X = laguerre_scipy(X, 3)
        
        y = sim.value[:, i+1]
        y = sim.params["rate_matrix"][i, i, itm] * y[itm] #discount to now
        
        t0 = time.time()
        #===================================================================================
        # model
        
        out = LinearRegression().fit(X, y.reshape(-1,1))
        pred= out.predict(X).reshape(-1,1)
        
        pred_all = np.zeros((training_paths , 1))
        pred_all[itm, :] = pred
        
        #===================================================================================
        t1 = time.time()
        total_time += (t1 - t0)
        
        
        best_results_r2["Step"].append(i)
        best_results_r2["Accuracy"].append(metrics.r2_score(out.predict(X).reshape(-1,1), y.reshape(-1,1)))
        
        sim.update(i, pred_all.ravel())
        
        verboseprint("Step: " + str(i))
        
        # =============================================================================
        # OOS Test
        # =============================================================================
        oos_r2["Step"].append(i)
        oos_r2["Accuracy"].append(oos_pred(berm_swaptions = sim_oos,
                                           model = out,
                                           step = i, laguerre = True)) #append r2 and update
        
    
    results_table = pd.DataFrame({"Step": steps,
                                 "Training R2": best_results_r2["Accuracy"],
                                 "OOS R2": oos_r2["Accuracy"],
                                  "Exc Pr": np.sum(sim.index[:, steps], axis = 0)/training_paths,
                                  "Exc Pr OOS": np.sum(sim_oos.index[:, steps], axis = 0)/testing_paths},
                                 )
    verboseprint(results_table)
    
    price = np.sum(sim.cmmf[:, :-1] * np.multiply(sim.index[:,1:], sim.value[:,1:]))/training_paths 
    verboseprint(price)
    
    price_oos = np.sum(sim_oos.cmmf[:, :-1] * np.multiply(sim_oos.index[:,1:], sim_oos.value[:,1:]))/testing_paths
    verboseprint(price_oos)
    
    return (price, price_oos, results_table, total_time)

### Run LR simulation for all rate paths contained in rate_list

In [37]:
lr_results_1_10 = []

for i in rate_list:
    lr_results_1_10.append(berm_lr(berm_swaptions = berm_swaptions,
                              sim_rates = i, strike = 0.0015, lockout = 1, tenor = 10))

## Bermudan Swaption Valuations with XGBoost
### Bayesian Opt using skopt
Bayesian optimization at each step \
Takes considerable amount of time to run ~1-2 hours depending on number of paths and steps estimated

In [38]:
def berm_xgb(berm_swaptions, sim_rates, strike, lockout, tenor, test_size = 0.1,
             search_iter = 15, cv = 3, search_scoring = 'neg_mean_squared_error',
             seed = 123, verbose = False):
    '''
    berm_swaptions: class
    sim_zeros : simulated zeros matrix n x n x paths
    strike: strike for the swaption
    lockout: in years
    tenor: in years
    test_size: float to split the dataset to training-testing split
    seed: seed for splitting
    '''
    verboseprint = print if verbose else lambda *a, **k: None
    
    total_dim = sim_rates.shape
    total_paths = total_dim[2] #no paths
    
    testing_paths = int(test_size * total_paths)
    training_paths = total_paths - testing_paths
    
    np.random.seed(123)
    testing_idx= np.sort(np.random.choice(np.arange(total_paths), testing_paths ,replace = False))
    training_idx = np.setdiff1d(np.arange(total_paths), testing_idx)
    
    sim = berm_swaptions(sim_rates[:, :, training_idx],
          strike = strike, tenor = tenor, lockout = lockout)
    
    #out of sample
    sim_oos = berm_swaptions(sim_rates[:, :, testing_idx],
                             strike = strike, tenor = tenor, lockout = lockout)
    
    steps = sim.exercisable_steps
    steps = np.flip(steps)[1:]
     
    model_list = []
    
    best_results_r2 = {"Step":[], "Accuracy":[]}
    oos_r2 = {"Step":[], "Accuracy":[]}
    
    total_time = 0
    for i in steps:
        '''
        #at each time step t_{M-1} compute V_{t_{M-1}} = max(h_{tm-1}, Q_{tm-1})
        #where Q is the expected continuation value
        #V will be used as an output for training the NN
        
        #first calculate Q at tm-1
        #which is the discounted value of G at tm
        #G is the approximated function from the neural network    
        '''
        itm = np.where(sim.intrinsic[:, i] > 0)[0]
        
        X = sim.X(i)[0]
        X = X[itm, :]
        
        X = laguerre_scipy(X, 3)
        
        y = sim.value[:, i+1]
        y = sim.params["rate_matrix"][i, i, itm] * y[itm] #discount to now
        
        t0 = time.time()
        #===================================================================================
        # model
        
        model = XGBRegressor()
        
        param_test = {
                'learning_rate': Real(0.01, 0.75, 'log-uniform'),
                'min_child_weight': Integer(0, 10, 'uniform'),
                'max_depth': Integer(3, 35, 'uniform'),
                'max_delta_step': Integer(0, 20),
                'subsample': Real(0.1, 1.0, 'uniform'),
                'colsample_bytree': Real(0.01, 1.0, 'uniform'),
                'colsample_bylevel': Real(0.01, 1.0, 'uniform'),
                'reg_lambda': Real(1e-9, 10, 'log-uniform'),
                'reg_alpha': Real(1e-9, 1e-2, 'log-uniform'),
                'gamma': Real(1e-9, 1e-3, 'log-uniform'), # minsplit loss
                'n_estimators': Integer(125, 350)
                }
        
        
        gsearch = BayesSearchCV(estimator = model, n_iter = search_iter,
                              search_spaces= param_test,
                              scoring= search_scoring, cv=cv, refit = True, random_state = seed)
        
        
        search_res = gsearch.fit(X, y)
        out = gsearch.best_estimator_ #XGBRegressor(**gsearch.best_params_).fit(X,y)
        
        model_list.append(gsearch.best_params_)
        
        pred = out.predict(X).reshape(-1,1)
        pred_all = np.zeros((training_paths, 1))
        pred_all[itm, :] = pred
        
        #===================================================================================
        t1 = time.time()
        total_time += t1 - t0
        
        best_results_r2["Step"].append(i)
        best_results_r2["Accuracy"].append(metrics.r2_score(out.predict(X).reshape(-1,1), y.reshape(-1,1)))
        
        #update
        sim.update(i, pred_all.ravel())
        
        verboseprint("Step: " + str(i))
        verboseprint(best_results_r2["Accuracy"][-1])
        
        # =============================================================================
        # OOS Test
        # =============================================================================
        oos_r2["Step"].append(i)
        #update
        oos_r2["Accuracy"].append(oos_pred(berm_swaptions = sim_oos,
                                           model = out,
                                           step = i, laguerre = True)) #append r2 and update, use laguerre expansion
    
    
    
    results_table = pd.DataFrame({"Step": np.flip(steps),
                                 "Training R2": best_results_r2["Accuracy"],
                                 "OOS R2": oos_r2["Accuracy"],
                                  "Exc Pr": np.sum(sim.index[:, np.flip(steps)], axis = 0)/training_paths,
                                  "Exc Pr OOS": np.sum(sim_oos.index[:, np.flip(steps)], axis = 0)/testing_paths},
                                 )
    
   
    verboseprint(results_table)
    
    price = np.sum(sim.cmmf[:, :-1] * np.multiply(sim.index[:,1:], sim.value[:,1:]))/training_paths 
    verboseprint(price)
    
    price_oos = np.sum(sim_oos.cmmf[:, :-1] * np.multiply(sim_oos.index[:,1:], sim_oos.value[:,1:]))/testing_paths
    verboseprint(price_oos)
    
    
    return (price, price_oos, results_table, total_time, model_list)

In [40]:
xgb_results_1_10 = []

for i in rate_list:
    xgb_results_1_10.append(berm_xgb(berm_swaptions = berm_swaptions,
                              sim_rates = i, strike = 0.0015, lockout = 1, tenor = 10, verbose = True))

Step: 18
0.9852836064910683
Step: 17
0.9949699280563477
Step: 16
0.995714533890012
Step: 15
0.9961352470847176
Step: 14
0.9962072898793132
Step: 13
0.9960149059095496
Step: 12
0.9949672179097943
Step: 11
0.9942850099635865
Step: 10
0.9959695496760629
Step: 9
0.7666032549031567
Step: 8
0.2174570012412309
Step: 7
-0.028553349402233863
Step: 6
-0.32595587531823433
Step: 5
0.6584164867068752
Step: 4
-0.8496527159560869
Step: 3
-1.8287200925857743
Step: 2
-1.4413601802637395
    Step  Training R2    OOS R2    Exc Pr  Exc Pr OOS
0      2     0.985284  0.962016  0.218889        0.17
1      3     0.994970  0.983059  0.122222        0.12
2      4     0.995715  0.949316  0.018889        0.02
3      5     0.996135  0.895410  0.075556        0.10
4      6     0.996207  0.771292  0.076667        0.02
5      7     0.996015  0.678754  0.010000        0.01
6      8     0.994967  0.416784  0.003333        0.01
7      9     0.994285 -0.014584  0.020000        0.00
8     10     0.995970  0.222887  0.0277

## Save run results

In [ ]:
#save run results
save_object(lr_results_1_10, path_valuation_results+"\\lr_results_1_10.pkl")

save_object(nn_results_1_10, path_valuation_results+"\\nn_results_1_10.pkl")

save_object(xgb_results_1_10, path_valuation_results+"\\xgb_results_1_10.pkl")